In [ ]:
import pandas as pd
import numpy as np
import datetime
import seaborn as sns
import talib
import plotly
import plotly.graph_objects as go
import plotly.offline as py
import math
from hyperopt import tpe, Trials, fmin
import hyperopt as hp
from geneticalgorithm import geneticalgorithm as ga
from binance.client import Client

In [ ]:
client = Client()
candles = client.get_klines(symbol='BNBBTC', interval=Client.KLINE_INTERVAL_30MINUTE)

In [ ]:
val_set = client.get_klines(symbol='ATOMBTC', interval=Client.KLINE_INTERVAL_15MINUTE, limit=1000)
val_set = pd.DataFrame(val_set)
val_set.columns = ['open_time','open','high','low','close','volume','close_time','quote_asset_volume','number_of_trades', 'a', 'b', 'c']
val_set = val_set[['open_time','close_time','open','high','low','close','volume','quote_asset_volume','number_of_trades']]
val_set['open'] = val_set['open'].astype(float)
val_set['high'] = val_set['high'].astype(float)
val_set['low'] = val_set['low'].astype(float)
val_set['close'] = val_set['close'].astype(float)

In [ ]:
from binance.client import Client

def import_data(symbol='ATOMBTC', limit=1000):
    
    client = Client()
    candles = client.get_klines(symbol=symbol, interval=Client.KLINE_INTERVAL_15MINUTE, limit=limit)
    
    df = pd.DataFrame(candles)
    df.columns = ['open_time','open','high','low','close','volume','close_time','quote_asset_volume','number_of_trades', 'a', 'b', 'c']
    df = df[['open_time','close_time','open','high','low','close','volume','quote_asset_volume','number_of_trades']]
    df['open'] = df['open'].astype(float)
    df['high'] = df['high'].astype(float)
    df['low'] = df['low'].astype(float)
    df['close'] = df['close'].astype(float)
    
    return df

df = import_data(symbol='ATOMBTC', limit=1000)

In [ ]:
df = import_data(symbol='ATOMBTC', limit=2000)

In [ ]:
df.shape

In [ ]:
bol = df.copy()
upper, middle, lower = talib.BBANDS(bol.open*100000, timeperiod=10, nbdevup=2, nbdevdn=2, matype=0)
bol['bol_upper'], bol['bol_middle'], bol['bol_lower'] = upper/100000, middle/100000, lower/100000
bol 

In [ ]:
def rsi_buy_ind(rsi, buy_threshold):      
    if rsi <= buy_threshold:
        return 1
    else:
        return 0

def rsi_sell_ind(rsi, sell_threshold):      
    if rsi >= sell_threshold:
        return 1 
    else:
        return 0
    
def macd_buy_ind(macd, buy_threshold):      
    if macd <= buy_threshold:
        return 1
    else:
        return 0

def macd_sell_ind(macd, sell_threshold):      
    if macd >= sell_threshold:
        return 1 
    else:
        return 0
    
def bol_buy_ind(price, bol_lower, buffer):      
    if price <= bol_lower * buffer:
        return 1
    else:
        return 0

def bol_sell_ind(price, bol_upper, buffer):      
    if price >= bol_upper * buffer:
        return 1 
    else:
        return 0

def buy_sell_strategy1(df, timeperiod, buy_threshold, sell_threshold):
    
    df_strategy = df.copy()
    
    df_strategy['rsi'] = talib.RSI(df_strategy.open, timeperiod=timeperiod)

    buy = np.vectorize(buy)
    sell = np.vectorize(sell)

    df_strategy['buy'] = buy(df_strategy['rsi'], buy_threshold)
    df_strategy['sell'] = sell(df_strategy['rsi'], sell_threshold)
    
    return df_strategy

def buy_sell_strategy2(df_in, 
                      rsi_timeperiod, rsi_buy_threshold, rsi_sell_threshold, w_rsi_buy, w_rsi_sell, 
                      macd_fastp, macd_slowp, macd_sigp, macd_buy_threshold, macd_sell_threshold, w_macd_buy, w_macd_sell, 
                      bol_period, bol_stdNbr, bol_buy_threshold, bol_sell_threshold, w_bol_buy, w_bol_sell, 
                      sell_point_execute, flexible_quantities, start_balance=0.001):
    
    buy_sigma_threshold = 0.7
    sell_sigma_threshold = 0.7
    
    df = df_in.reset_index().copy()
    
    df['price'] = df['open']
    df['balance'] = np.nan
    df.loc[0, 'balance'] = start_balance
    
    df['rsi'] = talib.RSI(df.price, timeperiod=rsi_timeperiod)
    df['macd'], df['macdsignal'], df['macdhist'] = talib.MACD(df.price, fastperiod=macd_fastp, slowperiod=macd_slowp, signalperiod=macd_sigp)    
    upper, middle, lower = talib.BBANDS(df.price*100000, timeperiod=bol_period, nbdevup=bol_stdNbr, nbdevdn=bol_stdNbr, matype=0)
    df['bol_upper'], df['bol_middle'], df['bol_lower'] = upper/100000, middle/100000, lower/100000
    
    df['rsi_buy_ind'] = np.vectorize(rsi_buy_ind)(df['rsi'], rsi_buy_threshold)
    df['macd_buy_ind'] = np.vectorize(macd_buy_ind)(df['macd'], macd_buy_threshold)
    df['bol_buy_ind'] = np.vectorize(bol_buy_ind)(df['price'], df['bol_lower'])
    df['rsi_sell_ind'] = np.vectorize(rsi_sell_ind)(df['rsi'], rsi_sell_threshold)
    df['macd_sell_ind'] = np.vectorize(macd_sell_ind)(df['macd'], macd_sell_threshold)
    df['bol_sell_ind'] = np.vectorize(bol_sell_ind)(df['price'], df['bol_upper'])
    
    bought = False
    stock_held = 0
    balance = start_balance
    sell_point_count = 0
    
    for i, pit in df.iterrows():
        if not bought:
            buy_sigma = w_rsi_buy * pit.rsi_buy_ind + w_macd_buy * pit.macd_buy_ind + w_bol_buy * pit.bol_buy_ind
            df.loc[i, 'buy_sigma'] = buy_sigma
            if buy_sigma >= buy_sigma_threshold:
                df.loc[i, 'buy'] = 1
                balance = balance - pit.price
                bought = True
            else:
                df.loc[i, 'buy'] = 0    

        else:
            sell_sigma = w_rsi_sell * pit.rsi_sell_ind + w_macd_sell * pit.macd_sell_ind + w_bol_sell * pit.bol_sell_ind
            df.loc[i, 'sell_sigma'] = sell_sigma
            if sell_sigma >= sell_sigma_threshold:
                sell_point_count = sell_point_count + 1
                if sell_point_count >= sell_point_execute:
                    df.loc[i, 'sell'] = 1
                    balance = balance + pit.price
                    bought = False
                    sell_point_count = 0
                else:
                    df.loc[i, 'sell'] = 0
            else:
                df.loc[i, 'sell'] = 0
                
        df.loc[i, 'balance'] = balance
    
    return balance, df

def buy_sell_strategy(df_in, 
                      rsi_timeperiod, rsi_buy_threshold, rsi_sell_threshold, w_rsi_buy, w_rsi_sell, 
                      macd_fastp, macd_slowp, macd_sigp, macd_buy_threshold, macd_sell_threshold, w_macd_buy, w_macd_sell, 
                      bol_period, bol_stdNbr, bol_buy_threshold, bol_sell_threshold, w_bol_buy, w_bol_sell, 
                      sell_point_execute, flexible_quantities, base_exchange_prop, start_balance=1):
    
    buy_sigma_threshold = 0.7
    sell_sigma_threshold = 0.7
    
    df = df_in.reset_index().copy()
    
    df['price'] = df['open']
    df['balance'] = np.nan
    df.loc[0, 'balance'] = start_balance
    
    df['rsi'] = talib.RSI(df.price, timeperiod=rsi_timeperiod)
    df['macd'], df['macdsignal'], df['macdhist'] = talib.MACD(df.price, fastperiod=macd_fastp, slowperiod=macd_slowp, signalperiod=macd_sigp)    
    upper, middle, lower = talib.BBANDS(df.price*100000, timeperiod=bol_period, nbdevup=bol_stdNbr, nbdevdn=bol_stdNbr, matype=0)
    df['bol_upper'], df['bol_middle'], df['bol_lower'] = upper/100000, middle/100000, lower/100000
    
    df['rsi_buy_ind'] = np.vectorize(rsi_buy_ind)(df['rsi'], rsi_buy_threshold)
    df['macd_buy_ind'] = np.vectorize(macd_buy_ind)(df['macd'], macd_buy_threshold)
    df['bol_buy_ind'] = np.vectorize(bol_buy_ind)(df['price'], df['bol_lower'], bol_buy_threshold)
    df['rsi_sell_ind'] = np.vectorize(rsi_sell_ind)(df['rsi'], rsi_sell_threshold)
    df['macd_sell_ind'] = np.vectorize(macd_sell_ind)(df['macd'], macd_sell_threshold)
    df['bol_sell_ind'] = np.vectorize(bol_sell_ind)(df['price'], df['bol_upper'], bol_sell_threshold)
    
    bought = False
    stock_held = 0
    balance = start_balance
    sell_point_count = 0
    
    #base_exchange = 100
    
    for i, pit in df.iterrows():
        base_exchange = (balance/pit.price) * base_exchange_prop
        buy_sigma = w_rsi_buy * pit.rsi_buy_ind + w_macd_buy * pit.macd_buy_ind + w_bol_buy * pit.bol_buy_ind
        df.loc[i, 'buy_sigma'] = buy_sigma
        if buy_sigma >= buy_sigma_threshold:
            if flexible_quantities:
                buy_quantity = base_exchange * buy_sigma/buy_sigma_threshold
            else:
                buy_quantity = base_exchange
            buy_quantity = min(buy_quantity, balance/pit.price)
            df.loc[i, 'buy'] = 1
            df.loc[i, 'buy_quantity'] = buy_quantity
            balance = balance - buy_quantity*pit.price
            bought = True
            stock_held = stock_held + buy_quantity
        else:
            df.loc[i, 'buy'] = 0
            df.loc[i, 'buy_quantity'] = 0
        sell_exchange = stock_held * base_exchange_prop
        sell_sigma = w_rsi_sell * pit.rsi_sell_ind + w_macd_sell * pit.macd_sell_ind + w_bol_sell * pit.bol_sell_ind
        df.loc[i, 'sell_sigma'] = sell_sigma
        if stock_held > 0:
            if sell_sigma >= sell_sigma_threshold:
                sell_point_count = sell_point_count + 1
                if flexible_quantities:
                    sell_quantity = sell_exchange * sell_sigma/sell_sigma_threshold
                else:
                    sell_quantity = sell_exchange                    
                sell_quantity = min(sell_quantity, stock_held)
                if sell_point_count >= sell_point_execute:
                    df.loc[i, 'sell'] = 1
                    df.loc[i, 'sell_quantity'] = sell_quantity
                    balance = balance + sell_quantity*pit.price
                    stock_held = stock_held - sell_quantity
                    bought = False
                    sell_point_count = 0
                else:
                    df.loc[i, 'sell'] = 0
                    df.loc[i, 'sell_quantity'] = 0
            else:
                df.loc[i, 'sell'] = 0
                df.loc[i, 'sell_quantity'] = 0
        else:
            df.loc[i, 'sell'] = 0
            df.loc[i, 'sell_quantity'] = 0
            
        df.loc[i, 'balance'] = balance
        df.loc[i, 'stock_held'] = stock_held
        
    df_final_balance = df.balance.iat[-1]
    df_final_stock_held = df.stock_held.iat[-1]
    df_final_price = df.price.iat[-1]
    
    portfolio_value = df_final_balance + df_final_price*df_final_stock_held
 
    if balance != df_final_balance:
        raise ValueError('Balance mismatch between algorithm ('+str(balance)+') and df record ('+str(df_final_balance)+')')
    if math.isnan(portfolio_value):
        print('df_final_balance: ' + str(df_final_balance))
        print('df_final_price: ' + str(df_final_price))
        print('df_final_stock_held: ' + str(df_final_stock_held))
        raise ValueError('portfolio_value is nan')
      
    return portfolio_value, df


In [ ]:
def objective_optimise_historic(params):

    prices = df.copy()
    
    rsi_timeperiod = params[0]
    rsi_buy_threshold = params[1]
    rsi_sell_threshold = params[2]
    w_rsi_buy = params[3]
    w_rsi_sell = params[4]
    
    macd_fastp = params[5]
    macd_slowp = params[6]
    macd_sigp = params[7]
    macd_buy_threshold = params[8]
    macd_sell_threshold = params[9]
    w_macd_buy = params[10]
    w_macd_sell = params[11]
    
    bol_period = params[12]
    bol_stdNbr = params[13]
    bol_buy_threshold = params[14]
    bol_sell_threshold = params[15]
    w_bol_buy = params[16]
    w_bol_sell = params[17]
    
    sell_point_execute = params[18]
    flexible_quantities = params[19]
    base_exchange_prop = params[20]
    
    if cross_validate:

        length = int(round(prices.shape[0]/5))
        cut1 = prices.iloc[:length, :]
        cut2 = prices.iloc[length:2*length, :]
        cut3 = prices.iloc[2*length:3*length, :]
        cut4 = prices.iloc[3*length:4*length, :]
        cut5 = prices.iloc[4*length:, :]
        
        cuts = [cut1, cut2, cut3, cut4, cut5]
        balances = []
        
        for cut in cuts:
            balance, prices_strategy = buy_sell_strategy(cut, rsi_timeperiod, rsi_buy_threshold, rsi_sell_threshold, w_rsi_buy, w_rsi_sell, 
                                                         macd_fastp, macd_slowp, macd_sigp, macd_buy_threshold, macd_sell_threshold, w_macd_buy, w_macd_sell, 
                                                         bol_period, bol_stdNbr, bol_buy_threshold, bol_sell_threshold, w_bol_buy, w_bol_sell,
                                                         sell_point_execute, flexible_quantities, base_exchange_prop)
            balances = balances + [balance]
            
        balance = np.mean(balances)
        
    else:
        
        balance, prices_strategy = buy_sell_strategy(prices, rsi_timeperiod, rsi_buy_threshold, rsi_sell_threshold, w_rsi_buy, w_rsi_sell, 
                                                     macd_fastp, macd_slowp, macd_sigp, macd_buy_threshold, macd_sell_threshold, w_macd_buy, w_macd_sell, 
                                                     bol_period, bol_stdNbr, bol_buy_threshold, bol_sell_threshold, w_bol_buy, w_bol_sell,
                                                     sell_point_execute, flexible_quantities, base_exchange_prop)

    return - balance
 

In [ ]:
def optimise_strategy(search_space, iterations, pop_size, max_iteration_without_improv=None):

    ga_param = {'max_num_iteration': iterations,
                'population_size': pop_size,
                'mutation_probability': 0.1,
                'elit_ratio': 0.01,
                'crossover_probability': 0.5,
                'parents_portion': 0.3,
                'crossover_type': 'uniform',
                'max_iteration_without_improv': max_iteration_without_improv}

    space_ga = np.array([search_space['rsi_timeperiod'],search_space['rsi_buy_threshold'],search_space['rsi_sell_threshold'],
                         search_space['w_rsi_buy'],search_space['w_rsi_sell'],search_space['macd_fastp'],search_space['macd_slowp'],
                         search_space['macd_sigp'],search_space['macd_buy_threshold'],search_space['macd_sell_threshold'],
                         search_space['w_macd_buy'],search_space['w_macd_sell'],search_space['bol_period'],search_space['bol_stdNbr'],
                         search_space['bol_buy_threshold'],search_space['bol_sell_threshold'],search_space['w_bol_buy'],
                         search_space['w_bol_sell'], search_space['sell_point_execute'], search_space['flexible_quantities'],
                         search_space['base_exchange_prop']])
    
    vartypes_ga = np.array([['int'],['real'],['real'],['real'],['real'],
                           ['int'],['int'],['int'],['real'],['real'],['real'],['real'],
                           ['int'],['int'],['real'],['real'],['real'],['real'],
                           ['int'],['int'], ['real']])

    best=ga(function=objective_optimise_historic, 
            dimension=len(space_ga), 
            variable_type_mixed=vartypes_ga,
            variable_boundaries=space_ga, 
            algorithm_parameters=ga_param)
    best.run()

    strategy = {'rsi_timeperiod': best.output_dict['variable'][0],
                'rsi_buy_threshold': best.output_dict['variable'][1],
                'rsi_sell_threshold': best.output_dict['variable'][2],
                'w_rsi_buy': best.output_dict['variable'][3],
                'w_rsi_sell': best.output_dict['variable'][4],
                'macd_fastp': best.output_dict['variable'][5],
                'macd_slowp': best.output_dict['variable'][6],
                'macd_sigp': best.output_dict['variable'][7],
                'macd_buy_threshold': best.output_dict['variable'][8],
                'macd_sell_threshold': best.output_dict['variable'][9],
                'w_macd_buy': best.output_dict['variable'][10],
                'w_macd_sell': best.output_dict['variable'][11],
                'bol_period': best.output_dict['variable'][12],
                'bol_stdNbr': best.output_dict['variable'][13],
                'bol_buy_threshold': best.output_dict['variable'][14],
                'bol_sell_threshold': best.output_dict['variable'][15],
                'w_bol_buy': best.output_dict['variable'][16],
                'w_bol_sell': best.output_dict['variable'][17],
                'sell_point_execute': best.output_dict['variable'][18],
                'flexible_quantities': best.output_dict['variable'][19],
                'base_exchange_prop': best.output_dict['variable'][20]}

    return strategy

In [ ]:
search_space = {'rsi_timeperiod': [2,28],
                'rsi_buy_threshold': [10,40],
                'rsi_sell_threshold': [50,80],
                'w_rsi_buy': [0, 1],
                'w_rsi_sell': [0, 1],
                'macd_fastp': [12, 12],
                'macd_slowp': [26, 26],
                'macd_sigp': [9, 9],
                'macd_buy_threshold': [0, 0],
                'macd_sell_threshold': [0, 0],
                'w_macd_buy': [0, 0],
                'w_macd_sell': [0, 0],
                'bol_period': [2, 20],
                'bol_stdNbr': [2, 6],
                'bol_buy_threshold': [1, 1],
                'bol_sell_threshold': [1, 1],
                'w_bol_buy': [0, 1],
                'w_bol_sell': [0, 1],
                'sell_point_execute': [1, 1],
                'flexible_quantities': [0, 0],
                'base_exchange_prop': [0.2, 0.2]
               }
cross_validate = False
optimal_strategy = optimise_strategy(search_space, iterations=6, pop_size=10, max_iteration_without_improv=30)

In [ ]:
def apply_strategy(df, strategy, plot=True):
    
    prices = df.copy()

    balance, applied_strategy = buy_sell_strategy(prices, 
                                                  strategy['rsi_timeperiod'],strategy['rsi_buy_threshold'],strategy['rsi_sell_threshold'],
                                                  strategy['w_rsi_buy'],strategy['w_rsi_sell'],strategy['macd_fastp'],strategy['macd_slowp'],
                                                  strategy['macd_sigp'],strategy['macd_buy_threshold'],strategy['macd_sell_threshold'],
                                                  strategy['w_macd_buy'],strategy['w_macd_sell'],strategy['bol_period'],strategy['bol_stdNbr'],
                                                  strategy['bol_buy_threshold'],strategy['bol_sell_threshold'],strategy['w_bol_buy'],
                                                  strategy['w_bol_sell'], strategy['sell_point_execute'], search_space['flexible_quantities'],
                                                  strategy['base_exchange_prop'])
    
    print('Final balance: ' + str(balance))
    
    if plot:
        buys = applied_strategy[applied_strategy.buy==1]
        sells = applied_strategy[applied_strategy.sell==1]

        Candle = go.Candlestick(x=applied_strategy.open_time,
                            open=applied_strategy.open,
                            high=applied_strategy.high,
                            low=applied_strategy.low,
                            close=applied_strategy.close
                            )

        Trace = go.Scatter(x=buys.open_time,
                           y=buys.open,
                           mode='markers',
                           name ='buy',
                           marker=go.scatter.Marker(size=10, color='blue', symbol='cross')
                           )

        Trace2 = go.Scatter(x=sells.open_time,
                           y=sells.open,
                           mode='markers',
                           name ='sell',
                           marker=go.scatter.Marker(size=10, color='black', symbol='cross')
                           )

        py.plot([Candle, Trace, Trace2])    

    return applied_strategy

In [ ]:
out = apply_strategy(df, optimal_strategy, plot=True)

In [ ]:
out

In [ ]:
out[out.sell==1]

In [ ]:
val = apply_strategy(val_set, optimal_strategy, plot=False)

In [ ]:
out.balance

In [ ]:
out.balance.iat[-1]

In [ ]:
'''
##add cross validation##
add stop loss
##skip first sell point##
add sell if gain x% order
add buy/sell quantity based on strength of indicators 
'''

In [ ]:
bayes_trials = Trials()
best = fmin(fn = objective_optimise_historic, space = space_opt, algo = tpe.suggest, max_evals = 200, trials = bayes_trials)

In [ ]:
optimal_strategy

In [ ]:
{'bol_buy_threshold': 0.0,
 'bol_period': 10.0,
 'bol_sell_threshold': 0.0,
 'bol_stdNbr': 2.0,
 'macd_buy_threshold': 0.0,
 'macd_fastp': 12.0,
 'macd_sell_threshold': 0.0,
 'macd_sigp': 9.0,
 'macd_slowp': 26.0,
 'rsi_buy_threshold': 19.807844365072651,
 'rsi_sell_threshold': 65.031637396731796,
 'rsi_timeperiod': 11.0,
 'w_bol_buy': 0.0,
 'w_bol_sell': 0.0,
 'w_macd_buy': 0.0,
 'w_macd_sell': 0.0,
 'w_rsi_buy': 1.0,
 'w_rsi_sell': 1.0}

In [ ]:
# 6.54% gains
strat = {'bol_buy_threshold': 0.0,
 'bol_period': 10.0,
 'bol_sell_threshold': 0.0,
 'bol_stdNbr': 2.0,
 'macd_buy_threshold': 0.0,
 'macd_fastp': 12.0,
 'macd_sell_threshold': 0.0,
 'macd_sigp': 9.0,
 'macd_slowp': 26.0,
 'rsi_buy_threshold': 25.418923746523333,
 'rsi_sell_threshold': 61.5440828284679,
 'rsi_timeperiod': 15.0,
 'w_bol_buy': 0.0,
 'w_bol_sell': 0.0,
 'w_macd_buy': 0.0,
 'w_macd_sell': 0.0,
 'w_rsi_buy': 1.0,
 'w_rsi_sell': 1.0,
 'sell_point_execute': 1.0}

In [ ]:
# 9.1% gains, low period RSI with sell order execution count
short_period_strat = {'bol_buy_threshold': 0.0,
 'bol_period': 10.0,
 'bol_sell_threshold': 0.0,
 'bol_stdNbr': 2.0,
 'macd_buy_threshold': 0.0,
 'macd_fastp': 12.0,
 'macd_sell_threshold': 0.0,
 'macd_sigp': 9.0,
 'macd_slowp': 26.0,
 'rsi_buy_threshold': 23.15940701875649,
 'rsi_sell_threshold': 73.163760739388707,
 'rsi_timeperiod': 2.0,
 'sell_point_execute': 4.0,
 'w_bol_buy': 0.0,
 'w_bol_sell': 0.0,
 'w_macd_buy': 0.0,
 'w_macd_sell': 0.0,
 'w_rsi_buy': 1.0,
 'w_rsi_sell': 1.0}

In [ ]:
# 15.7% gains, low period RSI with sell order execution count 2, cross validated
short_period_strat = {'bol_buy_threshold': 1.0,
 'bol_period': 15.0,
 'bol_sell_threshold': 1.0,
 'bol_stdNbr': 2.0,
 'flexible_quantities': 1.0,
 'macd_buy_threshold': 0.0,
 'macd_fastp': 12.0,
 'macd_sell_threshold': 0.0,
 'macd_sigp': 9.0,
 'macd_slowp': 26.0,
 'rsi_buy_threshold': 23.572504579146486,
 'rsi_sell_threshold': 52.693718967377684,
 'rsi_timeperiod': 3.0,
 'sell_point_execute': 2.0,
 'w_bol_buy': 0.79927168707409024,
 'w_bol_sell': 0.32621191375546432,
 'w_macd_buy': 0.0,
 'w_macd_sell': 0.0,
 'w_rsi_buy': 0.48791580460814421,
 'w_rsi_sell': 0.81777549294417684}